In [1]:
import torch.nn.functional as F
from torch.autograd import Variable
import copy
from collections import Counter
import torch 
import numpy as np
import torch.nn as nn

In [2]:
def read_data(path):
    f = open(path, "r")
    lines = f.readlines()
    f.close()
    data = []
    for l in lines:
        labelSplit = l.replace('\n','').split(' ', 1)
        data.append([labelSplit[0], [word.lower() for word in labelSplit[1].split()]])
    return data 

data = read_data('./questions.txt')

In [3]:
def remove_stop_words(data, path):
    stop_words = []
    with open('stop_words.txt') as f:
        stop_words = [word for line in f for word in line.split(",")]
    data_without_stop_words = []
    for k, v in data:
        words = [t for t in v if t not in stop_words]
        data_without_stop_words.append((k, words))
    return data_without_stop_words

data = remove_stop_words(data, './stop_words.txt')

In [4]:
def get_labels(data):
    _labels = []
    for k,v in data:
        _labels.append(k)   
    _unique_label = list(set(_labels))
    _unique_label_dict = {}
    for k,v in enumerate(_unique_label):
        _unique_label_dict[v] = k
    return _unique_label_dict

labels = get_labels(data)

In [200]:
def encode_labels(data, labels):
    encoded_labels = {}
    for i in range(0, 50): 
        label = [0] * 50
        label[i] = 1
        encoded_labels[i] = label
        
    cleaned_data = []
    for k,v in data:
        cleaned_data.append((encoded_labels[labels[k]],v))
        
    return cleaned_data

data = encode_labels(data, labels)

In [5]:
def create_indexed_vocab(data):
    vocab = []
    for _, sent in data:
        for word in sent:
            vocab.append(word)
    count = Counter(vocab)
    count = {w : count[w] for w in count if count[w] >= 2}
    vocab = []
    for k, v in count.items():
        vocab.append(k)
    indexed_vocab = {word: idx for idx, word in enumerate(vocab)}
    return indexed_vocab

indexed_vocab = create_indexed_vocab(data)
indexed_vocab['#UNK#'] = len(indexed_vocab)

In [8]:
def load_glove_embeddings(path, indexed_vocab, embedding_dim=300):
    with open(path) as f:
        embeddings = np.zeros((len(indexed_vocab), embedding_dim))
        for line in f.readlines():
            values = line.split()
            word = values[0]
            index = indexed_vocab.get(word)
            if index:
                vector = np.array(values[1:], dtype='float32')
                embeddings[index] = vector
            else:
                vector = np.array(values[1:], dtype='float32')
                embeddings[len(indexed_vocab) - 1] = vector
        return torch.from_numpy(embeddings).float()

glove = load_glove_embeddings('./glove.txt', indexed_vocab)
embeddings_random = nn.Embedding(glove.size(0), glove.size(1))
embeddings_pretrained = nn.Embedding.from_pretrained(glove, freeze=False)

In [11]:
def split_train_test(data, test_ratio):
    data_copy = copy.deepcopy(data)
    np.random.shuffle(data_copy)
    test_set_size = int(len(data) * test_ratio)
    test = data_copy[:test_set_size]
    train = data_copy[test_set_size:]
    return train, test

train, test = split_train_test(data, 0.1)

In [53]:
class BOWClassifier(nn.Module):
    def __init__(self,input_size,hidden_size, num_labels):
        super(BOWClassifier, self).__init__()
        self.input_size = input_size
        self.hidden_size  = hidden_size
        self.fc1 = torch.nn.Linear(self.input_size, self.hidden_size)
        self.relu = torch.nn.ReLU()
        self.fc2 = torch.nn.Linear(self.hidden_size,num_labels)
        #self.sigmoid = torch.nn.Sigmoid()
        
    def forward(self, x):
        hidden = self.fc1(x)
        relu = self.relu(hidden)
        output = self.fc2(relu)
        #output = self.sigmoid(output)
        return F.softmax(output)

In [54]:
input_size = 300
hidden_size = 10
num_labels = 50
bow = BOWClassifier(input_size, hidden_size, num_labels)

In [59]:
def make_bow_vector(sentence, indexed_vocab):
    vec = [0] * 300
    pt_tensor= torch.FloatTensor(vec)
    count = 0
    for word in sentence:
        count+=1
        if word not in indexed_vocab:
            pt_tensor = torch.add(pt_tensor, embeddings_pretrained(torch.LongTensor([indexed_vocab['#UNK#']]))[0])
        else:
            pt_tensor = torch.add(pt_tensor, embeddings_pretrained(torch.LongTensor([indexed_vocab[word]]))[0])
    pt_tensor=torch.div(pt_tensor,count)
    return pt_tensor

In [60]:
#loss_function = nn.MSELoss()
loss_function = nn.NLLLoss()
opt = torch.optim.SGD(bow.parameters(), lr = 0.1)

In [56]:

epoch = 2
for epoch in range(epoch):
    for l, d in data:
        opt.zero_grad()
        bow_vec = make_bow_vector(d, indexed_vocab) 
        y_pred = bow(bow_vec)
        ll=[]
        ll.append(labels[l])
        loss = loss_function(y_pred,ll)
        #print('Epoch {}: train loss: {}'.format(epoch, loss.item()))
        loss.backward()
        opt.step()

C:\Users\joshy\Anaconda3\lib\site-packages\ipykernel_launcher.py:16: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  app.launch_new_instance()


ValueError: Expected 2 or more dimensions (got 1)

In [117]:
#sample code
b=data[0:4]
print(b)
cdata=[]
target=[]
for l,d in b:
    bow_vec = make_bow_vector(d, indexed_vocab)
    cdata.append(list(bow_vec.tolist()))
    target.append(labels[l])
c_tensor= torch.FloatTensor(cdata)
t_tensor=torch.LongTensor(target)
y_pred = bow(c_tensor)
loss = loss_function(y_pred,t_tensor)
loss.backward()
opt.step()

[('ï»¿DESC:manner', ['serfdom', 'develop', 'leave', 'russia', '?']), ('ENTY:cremat', ['films', 'featured', 'character', 'popeye', 'doyle', '?']), ('DESC:manner', ['find', 'list', 'celebrities', "'", 'real', 'names', '?']), ('ENTY:animal', ['fowl', 'grabs', 'spotlight', 'chinese', 'year', 'monkey', '?'])]


C:\Users\joshy\Anaconda3\lib\site-packages\ipykernel_launcher.py:16: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  app.launch_new_instance()


In [ ]:
# batch_size =4 # or whatever
# epochs = 2
# for epoch in range(epochs):
#     for i in range(0,len(data), batch_size):
#         opt.zero_grad()
#         b=data[i:batch_size]
#         cdata=[]
#         target=[]
#         for l,d in b:
#             bow_vec = make_bow_vector(d, indexed_vocab)
#             cdata.append(list(bow_vec.tolist()))
#             target.append(labels[l])
#         c_tensor= torch.FloatTensor(cdata)
#         t_tensor=torch.LongTensor(target)
#         y_pred = bow(c_tensor)
#         loss = loss_function(y_pred,t_tensor)
#         loss.backward()
#         opt.step()